In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency, ttest_ind
from scipy import stats



In [2]:
errands_df = pd.read_parquet('Data/errands.parquet')
orders_df = pd.read_parquet('Data/orders.parquet')

In [3]:
errands_df = errands_df[errands_df['is_test_errand'] != 1]


In [4]:
currency_name_to_code = {
    'Euro': 'EUR',
    'Australian Dollar': 'AUD',
    'Brazilian Real': 'BRL',
    'US Dollar': 'USD',
    'Danish Krone': 'DKK',
    'Saudi Riyal': 'SAR',
    'Mexican Peso': 'MXN',
    'Pound Sterling': 'GBP',
    'Zloty': 'PLN',
    'Norwegian Krone': 'NOK',
    'Canadian Dollar': 'CAD',
    'United Arab Emirates dirham': 'AED',
    'Swedish Krona': 'SEK',
    'Chilean Peso': 'CLP',
    'Peso Uruguayo': 'UYU',
    'Nuevo Sol Peru': 'PEN',
    'South Korean Won': 'KRW',
    'Malaysian Ringgit': 'MYR',
    'Argentine Peso': 'ARS',
    'Thai Baht': 'THB',
    'Czech Koruna': 'CZK',
    'Colombian Peso Colombia': 'COP',
    'Kuwaiti Dinar': 'KWD',
    'Swiss Franc': 'CHF',
    'Hryvnia Ukraine': 'UAH',
    'South African Rand': 'ZAR',
    'Japanese yen': 'JPY',
    'Jordanian Dinar': 'JOD',
    'Bahraini Dinar': 'BHD',
    'New Zealand Dollar': 'NZD',
    'Indian Rupee': 'INR',
    'Egyptian Pound': 'EGP',
    'Bulgarian Lev': 'BGN',
    'Rupiah Indonesia': 'IDR',
    'Turkish Lira': 'TRY',
    'Qatari Rial': 'QAR',
    'Singapore Dollar': 'SGD',
    'Hong Kong Dollar': 'HKD',
    'Philippine Peso': 'PHP',
    'New Taiwan Dollar': 'TWD',
    'Rial Omani Oman': 'OMR',
    'Forint': 'HUF',
    'Yuan Renminbi': 'CNY',
    'Vietnamese dong': 'VND',
    'Iceland Krona': 'ISK',
    'Tenge Kazakhstan': 'KZT',
    'Uzbekistan Som': 'UZS'
}
#Static exchange rate, in actual analysis it has to be updated so it uses API, easy-exchange-rates and forex-python has been tried so far, 
#Read their documentationss to see why the exchange has problems. 
exchange_rates_to_usd = {
    'USD': 1,
    'EUR': 1.08,
    'AUD': 0.64,
    'BRL': 0.19,
    'DKK': 0.14,
    'SAR': 0.27,
    'MXN': 0.052,
    'GBP': 1.37,
    'PLN': 0.26,
    'NOK': 0.11,
    'CAD': 0.75,
    'AED': 0.27,
    'SEK': 0.095,
    'CLP': 0.0012,
    'UYU': 0.023,
    'PEN': 0.27,
    'KRW': 0.00076,
    'MYR': 0.23,
    'ARS': 0.010,
    'THB': 0.032,
    'CZK': 0.048,
    'COP': 0.00026,
    'KWD': 3.28,
    'CHF': 1.09,
    'UAH': 0.036,
    'ZAR': 0.055,
    'JPY': 0.0074,
    'JOD': 1.41,
    'BHD': 2.65,
    'NZD': 0.63,
    'INR': 0.013,
    'EGP': 0.032,
    'BGN': 0.58,
    'IDR': 0.000066,
    'TRY': 0.054,
    'QAR': 0.27,
    'SGD': 0.74,
    'HKD': 0.13,
    'PHP': 0.018,
    'TWD': 0.032,
    'OMR': 2.60,
    'HUF': 0.0031,
    'CNY': 0.15,
    'VND': 0.000043,
    'ISK': 0.0075,
    'KZT': 0.0023,
    'UZS': 0.000091
}



def convert_prices_to_usd(df):
    
    df['currency_code'] = df['currency'].map(currency_name_to_code)

    currency_index = df.columns.get_loc('currency')  
    df.insert(currency_index + 1, 'currency_code', df.pop('currency_code'))

    unmapped = df[df['currency_code'].isna()]['currency'].unique()
    if len(unmapped) > 0:
        print("Unmapped currencies:", unmapped)

    
    df['Order_Amount_in_usd'] = df.apply(
        lambda row: row['Order_Amount'] * exchange_rates_to_usd.get(row['currency_code'], 0) if row['currency_code'] in exchange_rates_to_usd else 0,
        axis=1
    )
    
    price_index = df.columns.get_loc('Order_Amount')
    df.insert(price_index + 1, 'Order_Amount_in_usd', df.pop('Order_Amount_in_usd'))

    df['Revenue_in_usd'] = df.apply(
        lambda row: row['Revenue'] * exchange_rates_to_usd.get(row['currency_code'], 0) if row['currency_code'] in exchange_rates_to_usd else 0,
        axis=1
    )
    
    price_index = df.columns.get_loc('Revenue')
    df.insert(price_index + 1, 'Revenue_in_usd', df.pop('Revenue_in_usd'))

    return df


orders_df = convert_prices_to_usd(orders_df)

In [19]:
orders_df['order_created_at'] = pd.to_datetime(orders_df['order_created_at'])
errands_df['created'] = pd.to_datetime(errands_df['created'])#.dt.date
errands_df['order_number'] = errands_df['order_number'].apply(lambda x: int(x, 36))
errands_df = errands_df.rename(columns={'order_number' : 'order_id'})

In [6]:
merged_df = pd.merge(orders_df, errands_df, on='order_id', how='inner')

In [7]:
merged_df['order_created_at'] = pd.to_datetime(merged_df['order_created_at'])#.dt.date
merged_df['created'] = pd.to_datetime(merged_df['created'])#.dt.date

# date_columns = ['created', 'order_created_at']

# for col in date_columns:
#     merged_df[col] = pd.to_datetime(merged_df[col]) 
#     merged_df[f'{col}_YearMonth'] = merged_df[col].dt.to_period('M')
    
#     # Get the index of the original column and move the new column
#     col_index = merged_df.columns.get_loc(col)
#     merged_df.insert(col_index + 1, f'{col}_YearMonth', merged_df.pop(f'{col}_YearMonth'))
    
    
#     merged_df.drop(col, axis=1, inplace=True)

In [ ]:
merged_df['Time_diff'] = ((merged_df['created'] - merged_df['order_created_at']).dt.total_seconds()) / 360*24

time_diff_stats = merged_df['Time_diff'].describe()
print(time_diff_stats)

merged_df['Time_diff'].hist(bins=50)
plt.title("Distribution of Time Between Order and Errand")
plt.xlabel("Time Difference (hours)")
plt.ylabel("Frequency")
plt.show()



In [64]:
merged_df['Time_diff_weeks'] = merged_df['Time_diff'] / ( 7) 
later_df = merged_df[merged_df['Time_diff_weeks'] > 1]

top_10_errands = later_df['errand_category'].value_counts().head(1)

top_10_errands_detailed = later_df.groupby('errand_category')['errand_type'].value_counts().head(1)

filtered_df = merged_df[merged_df['Time_diff_weeks'] < 1]

top_10 = filtered_df['errand_category'].value_counts().head(1)

top_10_detailed = filtered_df.groupby('errand_category')['errand_type'].value_counts().head(1)

print(f"Top 10 Errand Categories when time difference > 1 week: {top_10_errands}")
print(f"Top 10 Errand Categories when time difference < 1 week: {top_10}")

print(f"\nTop 10 Errand Types within each Category when time difference > 1 week: {top_10_errands_detailed}")
print(f"\nTop 10 Errand Types within each Category when time difference < 1 week: {top_10_detailed}")

Top 10 Errand Categories when time difference > 1 week: errand_category
2: Cancellation / refund    650562
Name: count, dtype: int64
Top 10 Errand Categories when time difference < 1 week: errand_category
6: Payment questions    1729
Name: count, dtype: int64

Top 10 Errand Types within each Category when time difference > 1 week: errand_category    errand_type
10: Force Majeure  2: Cancel      3503
Name: count, dtype: int64

Top 10 Errand Types within each Category when time difference < 1 week: errand_category   errand_type                             
11: Site support  1. No booking - general ETG site support    32
Name: count, dtype: int64


In [ ]:
canceled_orders = merged_df[merged_df['Is_Canceled'] == 1]
# canceled_orders.plot(kind='box', y='Order_Amount_in_usd', figsize=(6,6))
# sns.violinplot(x='Is_Canceled', y='Order_Amount_in_usd', data=merged_df)
canceled_orders['Order_Amount_in_usd'].plot(kind='hist', bins=30, figsize=(6,6))




In [11]:
errand_counts = merged_df.groupby('order_id').size().reset_index(name='errand_Count')

orders_with_errands = pd.merge(orders_df, errand_counts, on='order_id', how='left')

orders_with_errands['errand_Count'] = orders_with_errands['errand_Count'].fillna(0)

In [12]:
# # Define quantiles for categorization
# orders_with_errands['Order_Value_Category'] = pd.qcut(
#     orders_with_errands['Order_Amount_in_usd'], 
#     q=3,  # Change to 4 for quartiles or custom thresholds
#     labels=['Low', 'Medium', 'High']
# )


bins = [0, 100, 500, 1000, 5000, float('inf')]  
labels = ['Very Low', 'Low', 'Medium', 'High', 'Very High']

orders_with_errands['Order_Value_Category'] = pd.cut(
    orders_with_errands['Order_Amount_in_usd'], 
    bins=bins, 
    labels=labels, 
    include_lowest=True
)

In [13]:
value_analysis = orders_with_errands.groupby('Order_Value_Category').agg(
    Avg_Errand_Count=('errand_Count', 'mean'),
    Proportion_With_Errands=('errand_Count', lambda x: (x > 0).mean())
).reset_index()


/var/folders/_6/f4xb86d93znb83vn6wy29k2r0000gn/T/ipykernel_23522/193485439.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  value_analysis = orders_with_errands.groupby('Order_Value_Category').agg(


In [14]:

contingency_table = pd.crosstab(orders_with_errands['Order_Value_Category'], orders_with_errands['errand_Count'] > 0)
chi2, p, _, _ = chi2_contingency(contingency_table)

with_errands = orders_with_errands[orders_with_errands['errand_Count'] > 0]['Order_Amount_in_usd']
without_errands = orders_with_errands[orders_with_errands['errand_Count'] == 0]['Order_Amount_in_usd']
t_stat, p_val = ttest_ind(with_errands, without_errands, equal_var=False)


In [ ]:
sns.barplot(
    data=value_analysis, 
    x='Order_Value_Category', 
    y='Proportion_With_Errands'
)
plt.title('Proportion of Orders with Errands by Order Value')
plt.ylabel('Proportion with Errands')
plt.xlabel('Order Value Category')
plt.show()

In [ ]:
df = pd.merge(errands_df, orders_df, on='order_id', how='left')

currency_errands = df.groupby('Site_Country').size().reset_index(name='Errand_Count')

orders_per_currency = orders_df.groupby('Site_Country').size().reset_index(name='Order_Count')

currency_analysis = pd.merge(currency_errands, orders_per_currency, on='Site_Country', how='left')


currency_analysis['Errand_Rate'] = currency_analysis['Errand_Count'] / currency_analysis['Order_Count']

currency_analysis_sorted = currency_analysis.sort_values(by='Errand_Rate', ascending=False).head(10)

print(currency_analysis_sorted)

plt.figure(figsize=(10, 6))
plt.bar(currency_analysis_sorted['Site_Country'], currency_analysis_sorted['Errand_Rate'])
plt.xlabel('Country')
plt.ylabel('Errand Rate (Errands per Order)')
plt.title('Errand Rate by Country')
plt.xticks(rotation=45)
plt.show()

### Check if errands are more frequent for certain origin-destination pairs.

In [ ]:
errand_counts = merged_df.groupby(['Origin_Country', 'Destination_Country']).size().reset_index(name='Errand_Count')

order_counts = orders_df.groupby(['Origin_Country', 'Destination_Country']).size().reset_index(name='Order_Count')
merged_counts = pd.merge(errand_counts, order_counts, on=['Origin_Country', 'Destination_Country'])

merged_counts['Errand_Per_Order'] = merged_counts['Errand_Count'] / merged_counts['Order_Count']

merged_counts_sorted = merged_counts.sort_values(by='Errand_Per_Order', ascending=False)


merged_counts_sorted

In [ ]:
journey_types = ['One-way', 'Round-trip', 'OpenJawSingleOrigin', 'MultiStop',
       'OpenJawSingleDestination', 'OpenJawDouble'] 
filtered_df = merged_df[merged_df['Journey_Type_ID'].isin(journey_types)]

errands_count = filtered_df.groupby('order_id')['errand_id'].count().reset_index()
errands_count.rename(columns={'errand_id': 'Errand_count'}, inplace=True)

final_df = pd.merge(filtered_df, errands_count, on='order_id', how='left')

journey_analysis = final_df.groupby('Journey_Type_ID')['Errand_count'].mean().reset_index()

one_way_data = final_df[final_df['Journey_Type_ID'] == 'One-way']['Errand_count']
round_trip_data = final_df[final_df['Journey_Type_ID'] == 'Round-trip']['Errand_count']
multi_city_data = final_df[final_df['Journey_Type_ID'] == 'MultiStop']['Errand_count']

anova_result = stats.f_oneway(one_way_data, round_trip_data, multi_city_data)

print(journey_analysis)
print(f"ANOVA test result: F-statistic={anova_result.statistic}, p-value={anova_result.pvalue}")


In [ ]:
family_changes = merged_df[(merged_df['Customer_Group_Type'] =='FAMILY')]
# group_changes = merged_df[(merged_df['Is_Changed'] == 1 ) & (merged_df['Is_Canceled'] == 0 )]

# groupchanges = group_changes.groupby(['Customer_Group_Type', 'errand_type']).size().reset_index(name= 'Count').sort_values(by='Count', ascending=False)
familyChanges = family_changes.groupby('errand_category').size().reset_index(name= 'Count').sort_values(by='Count', ascending=False)
changes_by_category = family_changes.groupby('errand_type').size().reset_index(name='Count').sort_values(by='Count', ascending=False)

familyChanges.head()

In [77]:
# merged_df['Customer_Group_Type'].unique()

In [ ]:
main_changes = merged_df[(merged_df['Is_Changed'] == 1 ) & (merged_df['Is_Canceled'] == 0 )]
changes_by_type = main_changes.groupby('errand_type').size().reset_index(name='Count').sort_values(by='Count', ascending=False)
changes_by_category = main_changes.groupby('errand_category').size().reset_index(name='Count').sort_values(by='Count', ascending=False)

print(changes_by_type.head())
changes_by_category.head()

In [ ]:
main_canceled = merged_df[(merged_df['errand_category'] == '2: Cancellation / refund') ]
canceled_by_type = main_canceled.groupby('errand_type').size().reset_index(name='Count').sort_values(by='Count', ascending=False)
canceled_by_type.head()


In [80]:
errand_category = merged_df.groupby('errand_category').size().reset_index(name='Count').sort_values(by='Count', ascending=False)
# errand_category


In [81]:
X = merged_df[merged_df['Is_Canceled'] == 1]
# X['errand_category'].unique()

In [ ]:
canceled_value_counts = merged_df['errand_category'].value_counts()
threshold = canceled_value_counts.sum() * 0.02
canceled_counts = canceled_value_counts[canceled_value_counts > threshold]
other_count = merged_df[ ~merged_df['errand_category'].isin(canceled_counts.index)]['errand_category'].count()
canceled_counts['Other reasons'] = other_count
canceled_counts['Other reasons']

plt.figure(figsize=(10, 8))
canceled_counts.plot(
    kind='pie', 
    autopct='%1.1f%%',  
    startangle=140,     
    legend=False
)

plt.title('Distribution of Errand Categories', fontsize=16)
plt.ylabel('') 
plt.tight_layout()
plt.show()



In [ ]:
canceled_type_counts = merged_df[merged_df['errand_category'] == '2: Cancellation / refund']['errand_type'].value_counts()

threshold = canceled_type_counts.sum() * 0.01
canceled_counts = canceled_type_counts[canceled_type_counts > threshold]

other_count = merged_df[ ~merged_df['errand_type'].isin(canceled_counts.index)]['errand_type'].count()
canceled_counts['Other reasons'] = other_count
canceled_counts['Other reasons']
canceled_counts.reset_index(name='count')



In [84]:
errand_type_counts = merged_df[merged_df['errand_category'] == '1: Rebooking']['errand_type'].value_counts()

threshold = errand_type_counts.sum() * 0.01
errand_counts = errand_type_counts[errand_type_counts > threshold]

other_count = merged_df[ ~merged_df['errand_type'].isin(errand_counts.index)]['errand_type'].count()
errand_counts['Other reasons'] = other_count
errand_counts['Other reasons']
# errand_counts




np.int64(2057427)

In [85]:
errand_type_counts = merged_df[merged_df['errand_category'] == '18: Schedule change']['errand_type'].value_counts()

threshold = errand_type_counts.sum() * 0.01
errand_counts = errand_type_counts[errand_type_counts > threshold]

other_count = merged_df[ ~merged_df['errand_type'].isin(errand_counts.index)]['errand_type'].count()
errand_counts['Other reasons'] = other_count
errand_counts['Other reasons']
# errand_counts

np.int64(2228575)